In [3]:
!pip install datasets
!pip install audiomentations
!pip install transformers==4.11.3
!pip install librosa
!pip install jiwer

     |████████████████████████████████| 312 kB 290 kB/s            
     |████████████████████████████████| 212 kB 3.8 MB/s            
     |████████████████████████████████| 65 kB 308 kB/s            
     |████████████████████████████████| 2.9 MB 288 kB/s            
  Attempting uninstall: transformers
    Found existing installation: transformers 4.15.0
    Uninstalling transformers-4.15.0:
      Successfully uninstalled transformers-4.15.0


In [15]:
import torchaudio
import torch
import os
import matplotlib.pyplot as plt
import ipywidgets
import soundfile as sf
import torch
import datasets
import librosa
import random
import tensorflow as tf
import pandas as pd
import numpy as np
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift, AddBackgroundNoise, Gain
from typing import Tuple
from torch import Tensor
from torch.utils.data import Dataset
from IPython.display import Audio, display,HTML
from ipywidgets import IntProgress
from transformers import Wav2Vec2CTCTokenizer,Wav2Vec2ForCTC,Wav2Vec2Processor,Trainer,TrainingArguments,Wav2Vec2FeatureExtractor
from datasets import load_dataset, load_metric,Dataset,concatenate_datasets,set_caching_enabled, ClassLabel
from sklearn.model_selection import train_test_split

## AUGMENTING THE DATA

In [5]:
timit = load_dataset("timit_asr")
#timit = load_dataset("timit_asr")

Downloading:   0%|          | 0.00/2.40k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/869M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset timit_asr downloaded and prepared to /root/.cache/huggingface/datasets/timit_asr/clean/2.0.1/b11b576ddcccbcefa7c9f0c4e6c2a43756f3033adffe0fb686aa61043d0450ad. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
'''
train = {'train': timit[0]}
test = {'test': timit[1]}
from datasets import DatasetDict, Dataset
timit = DatasetDict()
# using your `Dict` object
for k,v in train.items():
    timit[k] = v
    
for k,v in test.items():
    timit[k] = v
'''

"\ntrain = {'train': timit[0]}\ntest = {'test': timit[1]}\nfrom datasets import DatasetDict, Dataset\ntimit = DatasetDict()\n# using your `Dict` object\nfor k,v in train.items():\n    timit[k] = v\n    \nfor k,v in test.items():\n    timit[k] = v\n"

In [7]:
timit

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'text', 'phonetic_detail', 'word_detail', 'dialect_region', 'sentence_type', 'speaker_id', 'id'],
        num_rows: 4620
    })
    test: Dataset({
        features: ['file', 'audio', 'text', 'phonetic_detail', 'word_detail', 'dialect_region', 'sentence_type', 'speaker_id', 'id'],
        num_rows: 1680
    })
})

In [8]:
waveform, sample_rate = torchaudio.load(timit['train']['audio'][5]['path'])

augment = Compose([
    AddGaussianNoise(min_amplitude=0.0003, max_amplitude=0.0025, p=0.7),
    # AddBackgroundNoise(min_snr_in_db=3,max_snr_in_db=30, noise_rms="relative"),
    TimeStretch(min_rate=0.8, max_rate=1.25, p=0.4),
    PitchShift(min_semitones=-4, max_semitones=4, p=0.8),
    Shift(min_fraction=-0.8, max_fraction=0.8, p=0.2),
])


# Augment/transform/perturb the audio data
augmented_samples = augment(samples=np.array(waveform), sample_rate=16000)


In [9]:
augmented_samples

array([[-2.8451390e-05, -3.0223579e-05,  3.9562674e-05, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00]], dtype=float32)

In [10]:
waveform[0]

tensor([-3.0518e-05, -3.0518e-05,  3.0518e-05,  ...,  3.0518e-05,
         0.0000e+00,  9.1553e-05])

In [11]:
print(timit['train'][5]["text"])
Audio(data=np.asarray(augmented_samples[0]), autoplay=True, rate=16000)

Ambidextrous pickpockets accomplish more.


In [12]:
print(timit['train'][5]["text"])
Audio(data=np.asarray(timit['train'][5]["audio"]["array"]), autoplay=True, rate=16000)

Ambidextrous pickpockets accomplish more.


In [47]:
##AUGMENT THE TRAINING DATA--> DICT FORMAT.
#W trnasform a DatasetDict to a dict object because it is faster to work with
# we will do the opposite later (dict to DatasetDict)
def dictionary_audio(dataset, augment):
    augment = Compose([
    AddGaussianNoise(min_amplitude=0.0003, max_amplitude=0.0025, p=0.8),
    Gain(min_gain_in_db=-15.0,max_gain_in_db=5.0, p=0.8),
    TimeStretch(min_rate=0.8, max_rate=1.25, p=0.8),
    ])

    dict_data = [{}] * len(dataset)

    for i in range(len(dataset)):
        waveform, sample_rate = torchaudio.load(dataset[i]['file'])
        if augment:
            waveform = augment(samples=np.array(waveform[0]),
                                sample_rate=16000)
        

        new_item = {'audio': {
            'wave': np.array(waveform),
            'sample_rate': sample_rate}
            }

        for k, v in dataset[i].items():
            if k != 'audio':
                new_item[k] = v

            dict_data[i] = new_item
    
    return dict_data  

In [20]:
augmented_data = dictionary_audio(timit['train'], augment=False)#clean train

In [48]:
test_data = dictionary_audio(timit['test'], augment=True)#dirdy test 

In [49]:
print(augmented_data[21]['text'])
Audio(data=np.asarray(augmented_data[21]['audio']['wave']), autoplay=True, rate=16000)

Curiosity and mediocrity seldom coexist.


In [50]:
print(test_data[32]['text'])
Audio(data=np.asarray(test_data[32]['audio']['wave']), autoplay=True, rate=16000)

The gunman kept his victim cornered at gunpoint for three hours.


### DATA

In [51]:
train = timit['train'] # CLEAN training data
test = test_data # NOISY test data

In [52]:
train[0]["audio"]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/404950a46da14eac65eb4e2a8317b1372fb3971d980d91d5d5b221275b1fd7e0/data/TRAIN/DR4/MMDM0/SI681.WAV',
 'array': array([-2.1362305e-04,  6.1035156e-05,  3.0517578e-05, ...,
        -3.0517578e-05, -9.1552734e-05, -6.1035156e-05], dtype=float32),
 'sampling_rate': 16000}

In [53]:
#test

#### Clean training data

In [54]:
print(train[4]['text'])
Audio(data=np.asarray(train[4]['audio']['array']), autoplay=True, rate=16000)

I honor my mom.


#### Noisy test data

In [55]:
print(test[3]['text'])
Audio(data=np.asarray(test[3]['audio']['wave']), autoplay=True, rate=16000)

She had your dark suit in greasy wash water all year.


In [56]:
timit = timit.remove_columns(["phonetic_detail", "word_detail", "dialect_region", "id", "sentence_type", "speaker_id"])

In [57]:
#display some random samples of the dataset
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))
    
show_random_elements(train.remove_columns(["audio", "file"]), num_examples=10)

,text,phonetic_detail,word_detail,dialect_region,sentence_type,speaker_id,id
0,She had your dark suit in greasy wash water all year.,"{'start': [0, 4236, 6440, 7937, 8960, 10600, 11320, 11995, 12920, 14040, 14280, 15826, 16600, 17800, 18000, 20469, 22276, 22760, 23018, 23661, 24563, 25820, 26360, 27112, 28359, 29320, 30935, 31925, 33796, 35800, 37375, 38135, 39101, 40392, 40814, 42321, 44076, 47126, 48003, 50025, 52114, 54680], 'stop': [4236, 6440, 7937, 8960, 10600, 11320, 11995, 12920, 14040, 14280, 15826, 16600, 17800, 18000, 20469, 22276, 22760, 23018, 23661, 24563, 25820, 26360, 27112, 28359, 29320, 30935, 31925, 33796, 35800, 37375, 38135, 39101, 40392, 40814, 42321, 44076, 47126, 48003, 50025, 52114, 54680, 57120], 'utterance': ['h#', 'sh', 'iy', 'hv', 'eh', 'dcl', 'jh', 'er', 'dcl', 'd', 'aa', 'r', 'kcl', 'k', 's', 'ux', 'tcl', 't', 'q', 'ih', 'n', 'gcl', 'g', 'r', 'iy', 's', 'iy', 'w', 'ao', 'sh', 'epi', 'w', 'ao', 'dx', 'axr', 'q', 'ao', 'l', 'y', 'ih', 'er', 'h#']}","{'start': [4236, 7937, 11320, 12920, 18000, 23018, 25820, 31925, 38135, 44076, 48003], 'stop': [7937, 11995, 12920, 18000, 23018, 25820, 31925, 37375, 42321, 48003, 54680], 'utterance': ['she', 'had', 'your', 'dark', 'suit', 'in', 'greasy', 'wash', 'water', 'all', 'year']}",DR4,SA,MLBC0,SA1
1,The cigarettes in the clay ashtray overflowed onto the oak table.,"{'start': [0, 2250, 2552, 2994, 4880, 6029, 6658, 6820, 8647, 9984, 10933, 11927, 12235, 12676, 13728, 13990, 14612, 15570, 16582, 17070, 19425, 21800, 23060, 23656, 24862, 25693, 27720, 28619, 29555, 30216, 31383, 33670, 34046, 35880, 36400, 37600, 38318, 38790, 39690, 40325, 41110, 42750, 43960, 45670, 46510, 47880, 50053, 51330], 'stop': [2250, 2552, 2994, 4880, 6029, 6658, 6820, 8647, 9984, 10933, 11927, 12235, 12676, 13728, 13990, 14612, 15570, 16582, 17070, 19425, 21800, 23060, 23656, 24862, 25693, 27720, 28619, 29555, 30216, 31383, 33670, 34046, 35880, 36400, 37600, 38318, 38790, 39690, 40325, 41110, 42750, 43960, 45670, 46510, 47880, 50053, 51330, 54800], 'utterance': ['h#', 'dh', 'ax', 's', 'ih', 'gcl', 'g', 'er', 'eh', 'tcl', 's', 'epi', 'ix', 'n', 'dh', 'ix', 'kcl', 'k', 'l', 'ey', 'ae', 'sh', 'tcl', 't', 'r', 'ey', 'q', 'ow', 'v', 'axr', 'f', 'l', 'ow', 'dx', 'ah', 'n', 'tcl', 't', 'ix', 'dh', 'iy', 'ow', 'kcl', 't', 'ey', 'q', 'el', 'h#']}","{'start': [2250, 2994, 12235, 13728, 14612, 19425, 28619, 36400, 40325, 42750, 45670], 'stop': [2994, 11927, 13728, 14612, 19425, 27720, 36400, 40325, 42750, 45670, 51330], 'utterance': ['the', 'cigarettes', 'in', 'the', 'clay', 'ashtray', 'overflowed', 'onto', 'the', 'oak', 'table']}",DR3,SX,MMAM0,SX247
2,I ate every oyster on Nora's plate.,"{'start': [0, 2911, 5875, 8840, 9480, 9720, 10567, 11780, 12407, 13061, 14370, 15715, 18310, 19800, 20060, 20710, 22892, 23880, 25399, 27112, 28574, 29869, 30788, 31958, 33170, 34030, 35046, 36804, 38100], 'stop': [2911, 5875, 8840, 9480, 9720, 10567, 11780, 12407, 13061, 14370, 15715, 18310, 19800, 20060, 20710, 22892, 23880, 25399, 27112, 28574, 29869, 30788, 31958, 33170, 34030, 35046, 36804, 38100, 39120], 'utterance': ['h#', 'ay', 'ey', 'tcl', 't', 'q', 'eh', 'v', 'r', 'iy', 'q', 'oy', 's', 'tcl', 't', 'er', 'q', 'ao', 'n', 'ao', 'r', 'ih', 'z', 'pcl', 'p', 'l', 'ey', 'tcl', 'h#']}","{'start': [2911, 5875, 9720, 15715, 23880, 25399, 31958], 'stop': [5875, 9720, 14370, 22892, 27112, 31958, 38100], 'utterance': ['i', 'ate', 'every', 'oyster', 'on', 'nora's', 'plate']}",DR3,SX,MDDC0,SX249
3,"The saw is broken, so chop the wood instead.","{'start': [0, 2040, 2506, 2966, 5054, 8355, 9480, 10680, 11543, 11920, 12600, 13960, 14920, 15369, 15982, 17371, 18240, 20172, 20840, 21514, 22956, 24520, 24920, 25320, 26605, 27440, 29120, 29800, 30200, 31193, 32383, 33520, 33789, 34339, 35880, 36440, 37000], 'stop': [2040, 2506, 2966, 5054, 8355, 9480, 10680, 11543, 11920, 12600, 13960, 14920, 15369, 15982, 17371, 18240, 20172, 20840, 21514, 22956, 24520, 24920, 25320, 26605, 27440, 29120, 29800, 30200, 31193, 32383, 33520,

### CREATE A VOCABOLARY

In [58]:
#We write a mapping function that concatenates all transcriptions into one long transcription and then 
#transforms the string into a set of chars. It is important to pass the argument batched=True to the map(...) 
#function so that the mapping function has access to all transcriptions at once.

import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    return batch

def extract_all_chars(batch):
    all_text = " ".join(batch["text"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}

In [59]:
timit = timit.map(remove_special_characters)

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

In [60]:
vocabs = timit.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=timit.column_names["train"])
vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

{'j': 0,
 "'": 1,
 'l': 2,
 'k': 3,
 'w': 4,
 's': 5,
 'f': 6,
 ' ': 7,
 'x': 8,
 'p': 9,
 'y': 10,
 'r': 11,
 'e': 12,
 'n': 13,
 'o': 14,
 'd': 15,
 't': 16,
 'i': 17,
 'b': 18,
 'c': 19,
 'u': 20,
 'v': 21,
 'm': 22,
 'g': 23,
 'z': 24,
 'q': 25,
 'a': 26,
 'h': 27}

In [61]:
#show_random_elements(librispeech_samples.remove_columns(["audio", "file"]), num_examples=10)

In [62]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)
                   

30

In [63]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [64]:
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2Processor, Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)
tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [65]:
augmented_data[0]["file"]

'/root/.cache/huggingface/datasets/downloads/extracted/404950a46da14eac65eb4e2a8317b1372fb3971d980d91d5d5b221275b1fd7e0/data/TRAIN/DR4/MMDM0/SI681.WAV'

In [66]:
augmented_data[0]["audio"]

{'wave': array([[-2.1362305e-04,  6.1035156e-05,  3.0517578e-05, ...,
         -3.0517578e-05, -9.1552734e-05, -6.1035156e-05]], dtype=float32),
 'sample_rate': 16000}

In [67]:
rand_int = random.randint(0, len(timit["train"]))

print("Target text:", timit["train"][rand_int]["text"])
print("Input array shape:", np.asarray(augmented_data[rand_int]["audio"]["wave"]).shape)
print("Sampling rate:", augmented_data[rand_int]["audio"]["sample_rate"])

Target text: the filtered air benefits allergies asthma sinus hay fever 
Input array shape: (1, 75879)
Sampling rate: 16000


### PREPARE THE DATESET FOR TRAINING 

In [68]:
#PREPARE THE TRAIN-CLEAN DATA----> for some reason, this code dosen't work if I create a function. 
inp = []
inp_l = []
for i in range(len(train)):

    # batched output is "un-batched" to ensure mapping is correct
    inputs = processor(train[i]['audio']["array"], sampling_rate=train[i]['audio']["sampling_rate"]).input_values
    input_lenght = len(inputs[0])

    inp.append(list(inputs))
    inp_l.append(input_lenght)


with processor.as_target_processor():
    labels = processor(timit['train']["text"]).input_ids
    
#getting the correct shape
shape = np.array(inp).shape[0]
inp = np.reshape(np.array(inp),(shape,))
dictt = {"input_values":list(inp), "input_length":inp_l, "labels": labels}


train_clean = {'train':dictt}

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [69]:
#PREPARE THE TEST-NOISY DATA
inp = []
inp_l = []
for i in range(len(test)):

    # batched output is "un-batched" to ensure mapping is correct
    inputs = processor(test[i]['audio']["wave"], sampling_rate=test[i]['audio']["sample_rate"]).input_values
    input_lenght = len(inputs[0])

    inp.append(list(inputs))
    inp_l.append(input_lenght)


with processor.as_target_processor():
    labels = processor(timit['test']["text"]).input_ids
    
#getting the correct shape
shape = np.array(inp).shape[0]
inp = np.reshape(np.array(inp),(shape,))
dict_test = {"input_values":list(inp), "input_length":inp_l, "labels": labels}

test_noisy = {'test':dict_test}

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [70]:
from datasets import Dataset, ClassLabel, Sequence, Features, Value

dataset = timit
# using your `Dict` object
for k,v in train_clean.items():
    dataset[k] = Dataset.from_dict(v)
    
for k,v in test_noisy.items():
    dataset[k] = Dataset.from_dict(v)


In [71]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input_values', 'input_length', 'labels'],
        num_rows: 4620
    })
    test: Dataset({
        features: ['input_values', 'input_length', 'labels'],
        num_rows: 1680
    })
})

In [72]:
np.array(dataset['train']["input_length"]).shape

(4620,)

In [73]:
max_input_length_in_sec = 4.0
dataset["train"] = dataset["train"].filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

  0%|          | 0/5 [00:00<?, ?ba/s]

In [74]:
np.array(dataset['train']["input_length"]).shape

(3978,)

#### Clean training data audio

In [75]:
Audio(data=np.asarray(dataset["train"][5]["input_values"]), autoplay=True, rate=16000)

#### Noisy test data audio

In [76]:
Audio(data=np.asarray(dataset["test"][5]["input_values"]), autoplay=True, rate=16000)

### TRAIN THE MODEL

In [77]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [78]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [79]:
wer_metric = load_metric("wer")

Downloading:   0%|          | 0.00/1.90k [00:00<?, ?B/s]

In [80]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [81]:
len(processor.tokenizer)

30

In [82]:
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    gradient_checkpointing=True,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

Downloading:   0%|          | 0.00/1.73k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2ForCTC: ['project_q.bias', 'project_hid.bias', 'quantizer.weight_proj.weight', 'project_q.weight', 'quantizer.codevectors', 'quantizer.weight_proj.bias', 'project_hid.weight']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to u

In [83]:
model.freeze_feature_extractor()

In [84]:
training_args = TrainingArguments(
  output_dir="./wav2vec2-large-xlsr-WOLOF",
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=40,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=3e-4,
  warmup_steps=1000,
  save_total_limit=2,
)

In [85]:
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=processor.feature_extractor,
)

In [86]:
dataset["train"].shape

(3978, 3)

In [87]:
dataset["test"].shape

(1680, 3)

In [88]:
os.environ['WANDB_MODE'] = 'dryrun'
os.environ['WANDB_DISABLED '] = 'True'

In [89]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length.
***** Running training *****
  Num examples = 3978
  Num Epochs = 40
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 4960
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


/opt/conda/lib/python3.7/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /usr/local/src/pytorch/aten/src/ATen/native/BinaryOps.cpp:461.)
  return torch.floor_divide(self, other)


Step,Training Loss,Validation Loss,Wer
500,3.653800,1.657518,0.993522
1000,0.528800,1.543618,0.603198
1500,0.210800,1.620449,0.612087
2000,0.135900,1.662506,0.589897
2500,0.096500,1.562823,0.591413
3000,0.070100,1.726094,0.592792
3500,0.057800,1.907086,0.594652
4000,0.048700,1.724128,0.584315
4500,0.038800,1.821248,0.584660


The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length.
***** Running Evaluation *****
  Num examples = 1680
  Batch size = 8
Saving model checkpoint to ./wav2vec2-large-xlsr-WOLOF/checkpoint-500
Configuration saved in ./wav2vec2-large-xlsr-WOLOF/checkpoint-500/config.json
Model weights saved in ./wav2vec2-large-xlsr-WOLOF/checkpoint-500/pytorch_model.bin
Configuration saved in ./wav2vec2-large-xlsr-WOLOF/checkpoint-500/preprocessor_config.json
The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length.
***** Running Evaluation *****
  Num examples = 1680
  Batch size = 8
Saving model checkpoint to ./wav2vec2-large-xlsr-WOLOF/checkpoint-1000
Configuration saved in ./wav2vec2-large-xlsr-WOLOF/checkpoint-1000/config.json
Model weights saved in ./wav2vec2-large-xlsr-WOLOF/checkpoint-1000/pytorch_model.bin
Configurat

TrainOutput(global_step=4960, training_loss=0.49116633149885364, metrics={'train_runtime': 14728.5673, 'train_samples_per_second': 10.803, 'train_steps_per_second': 0.337, 'total_flos': 1.3717006790225381e+19, 'train_loss': 0.49116633149885364, 'epoch': 40.0})

In [90]:
model.save_pretrained("wav2vec2-large-xlsr-WOLOF")
processor.save_pretrained("wav2vec2-large-xlsr-WOLOF")

Configuration saved in wav2vec2-large-xlsr-WOLOF/config.json
Model weights saved in wav2vec2-large-xlsr-WOLOF/pytorch_model.bin
Configuration saved in wav2vec2-large-xlsr-WOLOF/preprocessor_config.json
tokenizer config file saved in wav2vec2-large-xlsr-WOLOF/tokenizer_config.json
Special tokens file saved in wav2vec2-large-xlsr-WOLOF/special_tokens_map.json


In [91]:
model = Wav2Vec2ForCTC.from_pretrained("wav2vec2-large-xlsr-WOLOF").to("cuda")
processor = Wav2Vec2Processor.from_pretrained("wav2vec2-large-xlsr-WOLOF")

loading configuration file wav2vec2-large-xlsr-WOLOF/config.json
Model config Wav2Vec2Config {
  "_name_or_path": "facebook/wav2vec2-large-xlsr-53",
  "activation_dropout": 0.0,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForCTC"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 768,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": true,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "mean",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": true,
  "eos_token_id": 2,
  "feat_extract_activation": "gelu",
  "feat_extract_dropout": 0.0,
  "feat_extract_norm": "layer",
  "feat_proj_dropout": 0.0,
  "feat_quantizer_dropout": 0.0,
  "final_dropout": 0.0,
  "hidden_act": "g

In [92]:
def map_to_result(batch):
    with torch.no_grad():
        input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
        logits = model(input_values).logits

    pred_ids = torch.argmax(logits, dim=-1)
    batch["pred_str"] = processor.batch_decode(pred_ids)[0]
    batch["text"] = processor.decode(batch["labels"], group_tokens=False)

    return batch

In [93]:
results = dataset["test"].map(map_to_result, remove_columns=dataset["test"].column_names)

0ex [00:00, ?ex/s]

In [94]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["text"])))

Test WER: 0.500


In [95]:
show_random_elements(results)

,pred_str,text
0,am to balous her an oily benefet latage,aim to balance your employee benefit package
1,the pa regented him from a ritnulols mom,the fog prevented them from arriving on time
2,young children shold apoint sasus ton his vacious diseases,young children should avoid exposure to contagious diseases
3,art official intelligences worbel,artificial intelligence is for real
4,they'r false vershoes have viles share o all,their props were two stepladders a chair and a palm fan
5,if he wholwth shoers he be to poout,if people were more generous there would be no need for welfare
6,the fish began to eat prigtly on the surflace of the saliec,the fish began to leap frantically on the surface of the small lake
7,her ridehandakes whenever the barometeric pressure changes,her right hand aches whenever the barometric pressure changes
8,homly waire poup nanairs,only lawyers love millionaires
9,the nearost cinnogob may not be within walking distance,the nearest synagogue may not be within walking distance


In [96]:
model.to("cuda")

with torch.no_grad():
    logits = model(torch.tensor(dataset["test"][:1]["input_values"], device="cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)

# convert ids to tokens
" ".join(processor.tokenizer.convert_ids_to_tokens(pred_ids[0].tolist()))

"[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] t h e e | | [PAD] b [PAD] [PAD] [PAD] o n n [PAD] g [PAD] [PAD] a l l l [PAD] [PAD] [PAD] o o o [PAD] [PAD] [PAD] [PAD] [PAD] s s s [PAD] | | | [PAD] m [PAD] [PAD] [PAD] u u s s [PAD] [PAD] [PAD] n [PAD] t [PAD] | | [PAD] v [PAD] e [PAD] | | | s [PAD] [PAD] [PAD] [PAD] i [PAD] [PAD] t [PAD] [PAD] [PAD] u u [PAD] [PAD] [PAD] [PAD] a [PAD] t t e e | a d [PAD] | | [PAD] h h e e e a [PAD] r r r ' ' s s | | [PAD] s h h [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] o o r r [PAD] [PAD] [PAD] [PAD] |"